In [1]:
from datetime import date
from pathlib import Path
import getpass
import numpy
import pandas
import os
import sys

repo_path = str(Path().resolve().parent)
os.chdir(repo_path)
sys.path.append(repo_path)
from src.satellite_data.sentinel_data_downloader import SentinelDataDownloader
from src.satellite_data.models.sentinel_api_query_input import SentinelAPIQueryInput
from src.satellite_data.polygon_folder_manager.polygon_folder_manager import PolygonFolderManager

c:\Users\ETE008168\AppData\Local\anaconda3\envs\turbidity\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
USER = "phil190"
PASSWORD = getpass.getpass()
AREA_LIST = ["nb_bouctouche_cocagne", "ipe_dunk_west", "ipe_morell"]
YEAR_LIST = [2022, 2023]

In [5]:
%%capture
df = pandas.DataFrame(columns=["Area", "Year", "Nbr of Product", "Nbr of Product to Download", "Nbr of Online Product", "Nbr Product to Trigger", "Estimate Time to Trigger All"])
total_trigger_time = 0 
i = 0
for year in YEAR_LIST:
    for area in AREA_LIST:
        polygon_folder_manager = PolygonFolderManager(area)
        sentinel_api_query_input = SentinelAPIQueryInput(
            area=polygon_folder_manager.download_polygon,
            date=(date(year, 5, 1), date(year, 11, 1)),
            platformname="Sentinel-2",
            processinglevel="Level-2A",
            cloudcoverpercentage=(0, 100),
        )

        sentinel_data_downloader = SentinelDataDownloader(
            USER, PASSWORD, sentinel_api_query_input, polygon_folder_manager
        )
        sentinel_data_downloader._get_not_downloaded_uuid_isonline_dict()

        number_of_product = len(sentinel_data_downloader.products_df)
        number_of_product_to_download = len(sentinel_data_downloader.uuid_dict)
        number_of_online_product = numpy.array(list(sentinel_data_downloader.uuid_dict.values())).sum()
        number_of_product_to_trigger = number_of_product_to_download - number_of_online_product
        estimate_time_to_trigger_all_in_min = number_of_product_to_trigger*32
        estimate_time_to_trigger_all_in_hour_str = f"{int(estimate_time_to_trigger_all_in_min/60)}h:{estimate_time_to_trigger_all_in_min%60}m"
        total_trigger_time += estimate_time_to_trigger_all_in_min

        df.loc[i, "Area"] = area
        df.loc[i, "Year"] = year
        df.loc[i, "Nbr of Product"] = number_of_product
        df.loc[i, "Nbr of Product to Download"] = number_of_product_to_download
        df.loc[i, "Nbr of Online Product"] = number_of_online_product
        df.loc[i, "Nbr Product to Trigger"] = number_of_product_to_trigger
        df.loc[i, "Estimate Time to Trigger All"] = estimate_time_to_trigger_all_in_hour_str

        i += 1
df.loc[i, "Area"] = "All"
df.loc[i, "Year"] = '-'.join(str(year) for year in YEAR_LIST)
df.loc[i, "Nbr of Product"] = int(df.loc[:i, "Nbr of Product"].sum())
df.loc[i, "Nbr of Product to Download"] = int(df.loc[:i, "Nbr of Product to Download"].sum())
df.loc[i, "Nbr of Online Product"] = int(df.loc[:i, "Nbr of Online Product"].sum())
df.loc[i, "Nbr Product to Trigger"] = int(df.loc[:i, "Nbr Product to Trigger"].sum())
df.loc[i, "Estimate Time to Trigger All"] = f"{int(total_trigger_time/60)}h:{total_trigger_time%60}m"

In [6]:
df

,Area,Year,Nbr of Product,Nbr of Product to Download,Nbr of Online Product,Nbr Product to Trigger,Estimate Time to Trigger All
0,nb_bouctouche_cocagne,2022,73,0,0.0,0.0,0h:0.0m
1,ipe_dunk_west,2022,37,0,0.0,0.0,0h:0.0m
2,ipe_morell,2022,75,0,0.0,0.0,0h:0.0m
3,nb_bouctouche_cocagne,2023,40,0,0.0,0.0,0h:0.0m
4,ipe_dunk_west,2023,20,0,0.0,0.0,0h:0.0m
5,ipe_morell,2023,40,0,0.0,0.0,0h:0.0m
6,All,2022-2023,285,0,0,0,0h:0.0m
